In [7]:
#Nomenclature
RM = ['Tasla', 'Rangeen', 'Sponge', 'LocalScrap', 'ImportedScrap', 'HC', 'PigIron']

In [8]:
#Parameters
RM_Par =        {'Tasla':         {'Rate':17000,'Recovery':0.84,'Min':0,   'Max':0.5, 'Max/Month':800},
                 'Rangeen':       {'Rate':13600,'Recovery':0.74,'Min':0,   'Max':0.25,'Max/Month':500},
                 'Sponge':        {'Rate':17800,'Recovery':0.85,'Min':0.1, 'Max':0.5, 'Max/Month':1000},
                 'LocalScrap':    {'Rate':20000,'Recovery':0.94,'Min':0.15,'Max':0.8, 'Max/Month':1000},
                 'ImportedScrap': {'Rate':23000,'Recovery':0.97,'Min':0,   'Max':0.8, 'Max/Month':1500},
                 'HC':            {'Rate':2500, 'Recovery':0.25,'Min':0,   'Max':0.2, 'Max/Month':300},
                 'PigIron':       {'Rate':20400,'Recovery':0.95,'Min':0.05,'Max':0.1, 'Max/Month':500}}

In [9]:
#Defining Problem
import pulp as p
Lp_prob = p.LpProblem('Chandpur_Enterprise',p.LpMaximize)

In [10]:
#Decision Variables
RM_Var = p.LpVariable.dicts('Raw Material Quantities',[i for i in RM]) 
#ReEvaluate, this might lead to a problem, no lower bound defined, currently it is -infinity

In [11]:
#Objective Function
Lp_prob += (p.lpSum(RM_Par[i]['Recovery']*RM_Var[i] for i in RM)*29000)-p.lpSum(RM_Par[i]['Rate']*RM_Var[i] for i in RM)-((p.lpSum(RM_Var[i] for i in RM)*700+1200)*4.3)-2000*p.lpSum(RM_Par[i]['Recovery']*RM_Var[i] for i in RM)-3000

In [12]:
#Constraints
Lp_prob+= p.LpConstraint(e=p.LpAffineExpression([(RM_Var[i],RM_Par[i]['Recovery']) for i in RM]),sense=-1,name='Main',rhs=4)
for i in RM:
    Lp_prob+= RM_Par[i]['Min']*sum(RM_Var.values()) <= RM_Var[i]
for i in RM:
     Lp_prob+= RM_Var[i] <= RM_Par[i]['Max']*sum(RM_Var.values())
for i in RM:
    Lp_prob+= RM_Var[i] <= (RM_Par[i]['Max/Month']*(sum(RM_Var.values())*0.3+0.2))/600
    
#Lp_prob+= 0.84*RM_Var['Tasla']+ 0.74*RM_Var['Rangeen']+ 0.85*RM_Var['Sponge']+ 0.94*RM_Var['LocalScrap']+ 0.97*RM_Var['ImportedScrap']+ 0.25*RM_Var['HC']+ 0.95*RM_Var['PigIron'] <=4 

In [13]:
print(Lp_prob)

Chandpur_Enterprise:
MAXIMIZE
1240.0*Raw_Material_Quantities_HC + 180.0*Raw_Material_Quantities_ImportedScrap + 2370.0*Raw_Material_Quantities_LocalScrap + 2240.0*Raw_Material_Quantities_PigIron + 3370.0*Raw_Material_Quantities_Rangeen + 2140.0*Raw_Material_Quantities_Sponge + 2670.0*Raw_Material_Quantities_Tasla + -8160.0
SUBJECT TO
Main: 0.25 Raw_Material_Quantities_HC
 + 0.97 Raw_Material_Quantities_ImportedScrap
 + 0.94 Raw_Material_Quantities_LocalScrap
 + 0.95 Raw_Material_Quantities_PigIron + 0.74 Raw_Material_Quantities_Rangeen
 + 0.85 Raw_Material_Quantities_Sponge + 0.84 Raw_Material_Quantities_Tasla
 <= 4

_C1: - Raw_Material_Quantities_Tasla <= 0

_C2: - Raw_Material_Quantities_Rangeen <= 0

_C3: 0.1 Raw_Material_Quantities_HC
 + 0.1 Raw_Material_Quantities_ImportedScrap
 + 0.1 Raw_Material_Quantities_LocalScrap
 + 0.1 Raw_Material_Quantities_PigIron + 0.1 Raw_Material_Quantities_Rangeen
 - 0.9 Raw_Material_Quantities_Sponge + 0.1 Raw_Material_Quantities_Tasla <= 0

_C4: 0.

In [14]:
status = Lp_prob.solve()

In [15]:
print(p.LpStatus[status]) #The Solution Status

Optimal


In [16]:
Z= print(p.value(Lp_prob.objective))

5322.867561500001


In [17]:
for v in Lp_prob.variables():
    print (v.name,"=",v.varValue)
    

Raw_Material_Quantities_HC = 0.91397059
Raw_Material_Quantities_ImportedScrap = -0.0
Raw_Material_Quantities_LocalScrap = 0.81397059
Raw_Material_Quantities_PigIron = 0.27132353
Raw_Material_Quantities_Rangeen = 1.3566176
Raw_Material_Quantities_Sponge = 0.54264706
Raw_Material_Quantities_Tasla = 1.5279412
